# SQL query from table names - Continued

In [3]:
!pip install --quiet openai
from openai import OpenAI
import os
from google.colab import userdata

OPENAI_API_KEY  = userdata.get('OPENAI_API_KEY_Ironhack')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.5/325.5 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 2.5 MB/s eta 0:00:00


## The old Prompt

In [4]:
#The old prompt
old_context = [ {'role':'system', 'content':"""
you are a bot to assist in create SQL commands, all your answers should start with \
this is your SQL, and after that an SQL that can do what the user request. \
Your Database is composed by a SQL database with some tables. \
Try to maintain the SQL order simple.
Put the SQL command in white letters with a black background, and just after \
a simple and concise text explaining how it works.
If the user ask for something that can not be solved with an SQL Order \
just answer something nice and simple, maximum 10 words, asking him for something that \
can be solved with SQL.
"""} ]

old_context.append( {'role':'system', 'content':"""
first table:
{
  "tableName": "employees",
  "fields": [
    {
      "nombre": "ID_usr",
      "tipo": "int"
    },
    {
      "nombre": "name",
      "tipo": "varchar"
    }
  ]
}
"""
})

old_context.append( {'role':'system', 'content':"""
second table:
{
  "tableName": "salary",
  "fields": [
    {
      "nombre": "ID_usr",
      "type": "int"
    },
    {
      "name": "year",
      "type": "date"
    },
    {
      "name": "salary",
      "type": "float"
    }
  ]
}
"""
})

old_context.append( {'role':'system', 'content':"""
third table:
{
  "tablename": "studies",
  "fields": [
    {
      "name": "ID",
      "type": "int"
    },
    {
      "name": "ID_usr",
      "type": "int"
    },
    {
      "name": "educational_level",
      "type": "int"
    },
    {
      "name": "Institution",
      "type": "varchar"
    },
    {
      "name": "Years",
      "type": "date"
    }
    {
      "name": "Speciality",
      "type": "varchar"
    }
  ]
}
"""
})

## New Prompt.
We are going to improve it following the instructions of a Paper from the Ohaio University: [How to Prompt LLMs for Text-to-SQL: A Study in Zero-shot, Single-domain, and Cross-domain Settings](https://arxiv.org/abs/2305.11853). I recommend you read that paper.

For each table, we will define the structure using the same syntax as in a SQL create table command, and add the sample rows of the content.

Finally, at the end of the prompt, we'll include some example queries with the SQL that the model should generate. This technique is called Few-Shot Samples, in which we provide the prompt with some examples to assist it in generating the correct SQL.


In [5]:
context = [ {'role':'system', 'content':"""
 CREATE TABLE employees (
  ID_usr int,
  name varchar
);

-- Sample rows
INSERT INTO employees (ID_usr, name) VALUES
(1, 'John Doe'),
(2, 'Jane Smith');

CREATE TABLE salary (
  ID_usr int,
  year date,
  salary float
);

-- Sample rows
INSERT INTO salary (ID_usr, year, salary) VALUES
(1, '2023-01-01', 50000.00),
(2, '2023-01-01', 60000.00);

CREATE TABLE studies (
  ID int,
  ID_usr int,
  educational_level int,
  Institution varchar,
  Years date,
  Speciality varchar
);

-- Sample rows
INSERT INTO studies (ID, ID_usr, educational_level, Institution, Years, Speciality) VALUES
(1, 1, 5, 'XYZ University', '2015-01-01', 'Computer Science'),
(2, 2, 4, 'ABC College', '2016-01-01', 'Engineering');
"""} ]



In [6]:
#FEW SHOT SAMPLES
context.append( {'role':'system', 'content':"""
-- Example queries
-- 1. Retrieve all employees
SELECT * FROM employees;

-- 2. Get the salary of employee with ID 1 for year 2023
SELECT salary FROM salary WHERE ID_usr = 1 AND year = '2023-01-01';

-- 3. Find educational level and institution of employee with ID 2
SELECT educational_level, Institution FROM studies WHERE ID_usr = 2;
"""
})

In [7]:
#Functio to call the model.
def return_CCRMSQL(user_message, context):
    client = OpenAI(
    # This is the default and can be omitted
    api_key=OPENAI_API_KEY,
)

    newcontext = context.copy()
    newcontext.append({'role':'user', 'content':"question: " + user_message})

    response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=newcontext,
            temperature=0,
        )

    return (response.choices[0].message.content)

## NL2SQL Samples
We're going to review some examples generated with the old prompt and others with the new prompt.

In [8]:
#new
context_user = context.copy()
print(return_CCRMSQL("""
-- 1. Retrieve all employees' names
SELECT name FROM employees;
""", context_user))

```sql
SELECT name FROM employees;
```


In [9]:
#old
old_context_user = old_context.copy()
print(return_CCRMSQL("SELECT name FROM employees;", old_context_user))

This is your SQL:
```sql
SELECT name FROM employees;
```

Explanation: This SQL query selects the "name" column from the "employees" table, retrieving the names of all employees in the database.


In [10]:
#new
print(return_CCRMSQL("What is the average salary per institution?", context_user))

To calculate the average salary per institution, you can use the following SQL query:

```sql
SELECT s.Institution, AVG(s.salary) AS avg_salary
FROM salary s
JOIN employees e ON s.ID_usr = e.ID_usr
JOIN studies st ON s.ID_usr = st.ID_usr
GROUP BY s.Institution;
```

This query joins the `salary`, `employees`, and `studies` tables on the employee ID (`ID_usr`) and institution, then calculates the average salary per institution using the `AVG` function and groups the results by institution.


In [11]:
#old
print(return_CCRMSQL("What is the institution with the highest average salary?", old_context_user))

This is your SQL:
```sql
SELECT s.Institution, AVG(sa.salary) AS avg_salary
FROM studies s
JOIN employees e ON s.ID_usr = e.ID_usr
JOIN salary sa ON s.ID_usr = sa.ID_usr
GROUP BY s.Institution
ORDER BY avg_salary DESC
LIMIT 1;
```

This SQL query joins the tables `studies`, `employees`, and `salary` to calculate the average salary for each institution. It then selects the institution with the highest average salary by ordering the results in descending order and limiting the output to the top result.


# Exercise
 - Complete the prompts similar to what we did in class.
     - Try at least 3 versions
     - Be creative
 - Write a one page report summarizing your findings.
     - Were there variations that didn't work well? i.e., where GPT either hallucinated or wrong.
     - What did you learn?

In [17]:
context_v1 = [
    {'role': 'system', 'content': """
CREATE TABLE employees (
  ID_usr INT,
  name VARCHAR
);

-- Sample data for employees table
INSERT INTO employees (ID_usr, name) VALUES
(1, 'John Doe'),
(2, 'Jane Smith');

CREATE TABLE salary (
  ID_usr INT,
  year DATE,
  salary FLOAT
);

-- Sample data for salary table
INSERT INTO salary (ID_usr, year, salary) VALUES
(1, '2023-01-01', 50000.00),
(2, '2023-01-01', 60000.00);

CREATE TABLE studies (
  ID INT,
  ID_usr INT,
  educational_level INT,
  Institution VARCHAR,
  Years DATE,
  Speciality VARCHAR
);

-- Sample data for studies table
INSERT INTO studies (ID, ID_usr, educational_level, Institution, Years, Speciality) VALUES
(1, 1, 4, 'ABC College', '2010-01-01', 'Computer Science'),
(2, 2, 5, 'XYZ University', '2012-01-01', 'Engineering');
"""},

    {'role': 'system', 'content': """
-- Prompt with examples
-- Example queries for SQL generation

-- Example 1: Select all columns from employees table
SELECT * FROM employees;

-- Example 2: Select employee name from employees where ID_usr = 1
SELECT name FROM employees WHERE ID_usr = 1;

-- Example 3: Select salary from salary where ID_usr = 1 and year = '2023-01-01'
SELECT salary FROM salary WHERE ID_usr = 1 AND year = '2023-01-01';

-- Example 4: Select educational_level, Institution from studies where ID_usr = 2
SELECT educational_level, Institution FROM studies WHERE ID_usr = 2;

-- Example 5: Join employees and salary to get name and salary
SELECT e.name, s.salary FROM employees e JOIN salary s ON e.ID_usr = s.ID_usr;

-- Example 6: Count employees with educational_level = 5
SELECT COUNT(*) AS num_employees FROM employees e JOIN studies s ON e.ID_usr = s.ID_usr WHERE s.educational_level = 5;

-- Example 7: Calculate average salary by educational level
SELECT s.educational_level, AVG(s.salary) AS avg_salary FROM salary s JOIN studies st ON s.ID_usr = st.ID_usr GROUP BY st.educational_level;
"""}
]


In [20]:
print(return_CCRMSQL("What is the average salary per institution?", context_v1))

To calculate the average salary per institution, we need to join the `employees`, `salary`, and `studies` tables based on the `ID_usr` column. Then, we can group the results by the `Institution` column and calculate the average salary for each institution. Here is the SQL query for this:

```sql
SELECT st.Institution, AVG(s.salary) AS avg_salary
FROM employees e
JOIN salary s ON e.ID_usr = s.ID_usr
JOIN studies st ON e.ID_usr = st.ID_usr
GROUP BY st.Institution;
```

This query will provide you with the average salary for each institution based on the data in the tables.


This version clearly defines table structures (employees, salary, studies) and includes sample rows, aiding the model's understanding of data. It provides basic example queries (SELECT * FROM employees, etc.) that are straightforward and aligned with the provided data structures.
However, it focuses on simple queries, potentially limiting the model's ability to handle more complex SQL tasks like joins and aggregations effectively.

In [18]:
context_v2 = [
    {'role': 'system', 'content': """
CREATE TABLE employees (
  ID_usr INT,
  name VARCHAR
);

-- Sample data for employees table
INSERT INTO employees (ID_usr, name) VALUES
(1, 'John Doe'),
(2, 'Jane Smith');

CREATE TABLE salary (
  ID_usr INT,
  year DATE,
  salary FLOAT
);

-- Sample data for salary table
INSERT INTO salary (ID_usr, year, salary) VALUES
(1, '2023-01-01', 50000.00),
(2, '2023-01-01', 60000.00);

CREATE TABLE studies (
  ID INT,
  ID_usr INT,
  educational_level INT,
  Institution VARCHAR,
  Years DATE,
  Speciality VARCHAR
);

-- Sample data for studies table
INSERT INTO studies (ID, ID_usr, educational_level, Institution, Years, Speciality) VALUES
(1, 1, 4, 'ABC College', '2010-01-01', 'Computer Science'),
(2, 2, 5, 'XYZ University', '2012-01-01', 'Engineering');
"""},

    {'role': 'system', 'content': """
-- Prompt with examples
-- Example queries for SQL generation

-- Example 1: Select all columns from employees table
SELECT * FROM employees;

-- Example 2: Select employee name from employees where ID_usr = 1
SELECT name FROM employees WHERE ID_usr = 1;

-- Example 3: Select salary from salary where ID_usr = 1 and year = '2023-01-01'
SELECT salary FROM salary WHERE ID_usr = 1 AND year = '2023-01-01';

-- Example 4: Select educational_level, Institution from studies where ID_usr = 2
SELECT educational_level, Institution FROM studies WHERE ID_usr = 2;

-- Example 5: Join employees and salary to get name and salary
SELECT e.name, s.salary FROM employees e JOIN salary s ON e.ID_usr = s.ID_usr;

-- Example 6: Count employees with educational_level = 5
SELECT COUNT(*) AS num_employees FROM employees e JOIN studies s ON e.ID_usr = s.ID_usr WHERE s.educational_level = 5;

-- Example 7: Calculate average salary by educational level
SELECT s.educational_level, AVG(s.salary) AS avg_salary FROM salary s JOIN studies st ON s.ID_usr = st.ID_usr GROUP BY st.educational_level;
"""}
]


In [21]:
print(return_CCRMSQL("What is the average salary per institution?", context_v2))

To calculate the average salary per institution, we need to join the `employees`, `salary`, and `studies` tables based on the `ID_usr` column. Then, we can group the results by the `Institution` column and calculate the average salary for each institution. Here is the SQL query for this:

```sql
SELECT st.Institution, AVG(s.salary) AS avg_salary
FROM employees e
JOIN salary s ON e.ID_usr = s.ID_usr
JOIN studies st ON e.ID_usr = st.ID_usr
GROUP BY st.Institution;
```

This query will provide you with the average salary for each institution based on the data in the `employees`, `salary`, and `studies` tables.


This version extends the prompt with more complex queries involving joins (JOIN) and aggregations (COUNT, AVG), enhancing the model's capabilities to handle diverse SQL tasks.
Provides comprehensive few-shot examples that cover a wider range of SQL operations, including data aggregation and analysis.
But it has an increased complexity which may challenge the model, leading to potential errors or incorrect SQL generation, especially for nuanced queries requiring precise logic.




Both `context_v1` and `context_v2` produced identical SQL queries for the given question. This consistency suggests that the improvements made to both contexts were effective in guiding the model to generate the correct SQL structure.

The fact that both contexts yielded the correct SQL query indicates that the model comprehended the prompt well and utilized the provided schema information (`employees`, `salary`, `studies` tables) effectively to generate the desired SQL.

The clarity of the prompt contexts (`context_v1` and `context_v2`) was sufficient to direct the model in understanding the database schema, relationships, and operations required to answer the query accurately. This suggests that the improvements made, such as explicit table definitions, sample data, and example queries, contributed positively to the model's performance.

The test case "What is the average salary per institution?" effectively validated that both contexts produced correct SQL outputs. This verifies that the contexts are appropriately structured and informative for guiding SQL generation tasks.

Both versions allow the model to generate SQL queries based on provided contexts and example queries. Version 2 demonstrates better coverage of complex SQL scenarios but may also lead to more errors or inaccuracies due to its increased complexity.
